In [3]:
import onnxruntime    # to inference ONNX models, we use the ONNX Runtime
import onnx

In [4]:
session = onnxruntime.InferenceSession('mobilenet.onnx', None)

In [5]:
input_name = session.get_inputs()[0].name  

In [6]:
input_name

'input.1'

In [7]:
def preprocess(input_data):
    # convert the input data into the float32 input
    img_data = input_data.astype('float32')

    #normalize
    mean_vec = np.array([0.485, 0.456, 0.406])
    stddev_vec = np.array([0.229, 0.224, 0.225])
    norm_img_data = np.zeros(img_data.shape).astype('float32')
    for i in range(img_data.shape[0]):
        norm_img_data[i,:,:] = (img_data[i,:,:]/255 - mean_vec[i]) / stddev_vec[i]

    #add batch channel
    norm_img_data = norm_img_data.reshape(1,3,224, 224).astype('float32')
    return norm_img_data

In [8]:
from PIL import Image, ImageDraw, ImageFont, ImageOps

In [22]:
image = Image.open('images/caesar_salad/1000016.jpg')

In [23]:
import numpy as np

In [24]:
imnew=ImageOps.fit(image, (224,224))
image_data = np.array(imnew).transpose(2, 0, 1)
input_data = preprocess(image_data)

In [16]:
def softmax(x):
    x = x.reshape(-1)
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [17]:
def postprocess(result):
    return softmax(np.array(result)).tolist()

In [25]:
import time
start = time.time()
raw_result = session.run([], {input_name: input_data})
end = time.time()
res = postprocess(raw_result)

inference_time = np.round((end - start) * 1000, 2)
idx = np.argmax(res)

In [28]:
res[idx]

0.9999892711639404